# 1. Import Dependencies

In [41]:
import os
import cv2
import random
import numpy as np
import pandas as pd

# 2. Data Preparation

In [42]:
feature_vector = []
target_vector = []

df = pd.read_csv("../../datasets/faces/faces.csv")

# Remove images with more than one face
df = df.groupby('image_name').filter(lambda x: len(x) == 1).reset_index(drop=True)

# Shuffle the data frame
df = df.sample(frac=1).reset_index(drop=True)

FD_FRAME_SIZE = 640
DATASET_DIR = "../../datasets/faces/images"

win_size = (FD_FRAME_SIZE, FD_FRAME_SIZE) 
block_size = (32, 32)
block_stride = (32, 32)
cell_size = (8, 8)
num_bins = 9

HOG = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)


In [43]:
for index, image_properties in df.iterrows():

    image_path = os.path.join(DATASET_DIR, image_properties["image_name"])

    gray_image = cv2.resize(cv2.imread(image_path, 0), win_size)

    x0 = int(image_properties["x0"] / image_properties["width"])
    x1 = int(image_properties["x1"] / image_properties["width"])
    y0 = int(image_properties["y0"] / image_properties["height"])
    y1 = int(image_properties["y1"] / image_properties["height"])

    normalized_coordinate_vector = [x0, y0, x1, y1]

    hog = HOG.compute(gray_image).flatten()
    normalized_hog = hog / hog.max()

    feature_vector.append(normalized_hog)
    target_vector.append(normalized_coordinate_vector)

    # bgr_image[y0:y1, x0:x1, 1] = 255

    # if gray_image is not None:
    #     cv2.imshow("Image", gray_image)
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()

    # if index == 10:
    #     break

# 3. Data Split

In [44]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature_vector, target_vector, test_size=0.05, random_state=42)

# 4. Training

In [45]:
from sklearn.linear_model import LinearRegression

face_detector_model = LinearRegression().fit(x_train, y_train)

# 5. Evaluation

In [46]:
from sklearn.metrics import mean_squared_error

y_predicted = face_detector_model.predict(x_test)

mse = mean_squared_error(y_test, y_predicted)

print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.001576518622949081


# 6. Image Testing

In [49]:
test_image = cv2.imread(os.path.join(DATASET_DIR, random.choice(os.listdir(DATASET_DIR))))

gray_test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
test_feature = HOG.compute(cv2.resize(gray_test_image, win_size))
test_feature /= test_feature.max()

[[x0p, y0p, x1p, y1p]] = face_detector_model.predict([test_feature])

x0p = int(x0p * test_image.shape[0])
x1p = int(x1p * test_image.shape[0])
y0p = int(y0p * test_image.shape[1])
y1p = int(y1p * test_image.shape[1])

test_image[y0p:y1p, x0p:x1p, 1] = 255

if test_image is not None:
    cv2.imshow("Testing", test_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 7. Real-Time Testing

In [59]:
# Open the default camera (camera index 0)
camera = cv2.VideoCapture(0)

while camera.isOpened():
    # Capture frame-by-frame
    ret, frame = camera.read()

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_features = HOG.compute(cv2.resize(gray_frame, win_size))
    normalized_frame_features = frame_features / frame_features.max()

    [[x0f, y0f, x1f, y1f]] = face_detector_model.predict([normalized_frame_features])

    x0f = int(x0f * frame.shape[0])
    x1f = int(x1f * frame.shape[0])
    y0f = int(y0f * frame.shape[1])
    y1f = int(y1f * frame.shape[1])

    frame[y0f:y1f, x0f:x1f, 1] = 255

    cv2.imshow('Face Detector', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print(frame.shape)
        break

# Release the video capture object and close the window
camera.release()
cv2.destroyAllWindows()

(480, 640, 3)


# 8. Save the Model

In [60]:
import joblib

joblib.dump(face_detector_model, "Face_Detecting_Model_Trial_I.pkl")

['Face_Detecting_Model_Trial_I.pkl']